Screen Time Post Processing
---


In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series, ExcelFile, ExcelWriter
import datetime
import time

### Phase 1: Load Screen Timing for Subject 1 and Subject 2

In [3]:
phase_0_rows = 3*2
phase_1_rows = 12*2
phase_2_rows = 2*2
phase_3_rows = 11*2

total_rows = phase_0_rows + phase_1_rows + phase_2_rows + phase_3_rows
total_rows

56

In [4]:
subject_1_timing = pd.read_csv('data/timing_subject_1.csv', skipinitialspace=True, parse_dates=['time'])
assert len(subject_1_timing) == total_rows

In [5]:
subject_2_timing = pd.read_csv('data/timing_subject_2.csv', skipinitialspace=True, parse_dates=['time'])
assert len(subject_2_timing) == total_rows

In [6]:
screen_timing = subject_1_timing.append(subject_2_timing)
screen_timing = screen_timing.reset_index(drop=True)

In [7]:
screen_timing.rename(columns={'subject':'SubjectID', 'phase':'Phase'}, inplace=True)

In [8]:
screen_timing.head()

,SubjectID,state,Phase,time
0,1,start,0,2018-01-19 17:52:42.999642
1,1,stop,0,2018-01-19 17:52:55.195631
2,1,start,0,2018-01-19 17:53:02.695930
3,1,stop,0,2018-01-19 17:53:15.302756
4,1,start,0,2018-01-19 17:53:22.896308


In [9]:
screen_timing.time = screen_timing.time.dt.tz_localize('utc')
screen_timing.time = screen_timing.time.dt.tz_convert('America/New_York')
screen_timing.head()

,SubjectID,state,Phase,time
0,1,start,0,2018-01-19 12:52:42.999642-05:00
1,1,stop,0,2018-01-19 12:52:55.195631-05:00
2,1,start,0,2018-01-19 12:53:02.695930-05:00
3,1,stop,0,2018-01-19 12:53:15.302756-05:00
4,1,start,0,2018-01-19 12:53:22.896308-05:00


In [10]:
num_screens = len(screen_timing)
start_times = screen_timing.loc[[i for i in range(0, num_screens, 2)], 'time'].reset_index(drop=True)
stop_times = screen_timing.loc[[i for i in range(1, num_screens, 2)], 'time'].reset_index(drop=True)

In [11]:
screen_timing.drop('time', axis=1, inplace=True)
screen_timing.drop('state', axis=1, inplace=True)

In [12]:
screen_timing = screen_timing.loc[[i for i in range(0, num_screens, 2)]]
screen_timing = screen_timing.reset_index(drop=True)

In [13]:
screen_timing['Start'] = start_times
screen_timing['Stop'] = stop_times

In [14]:
screen_timing.head()

,SubjectID,Phase,Start,Stop
0,1,0,2018-01-19 12:52:42.999642-05:00,2018-01-19 12:52:55.195631-05:00
1,1,0,2018-01-19 12:53:02.695930-05:00,2018-01-19 12:53:15.302756-05:00
2,1,0,2018-01-19 12:53:22.896308-05:00,2018-01-19 12:53:34.942321-05:00
3,1,1,2018-01-19 12:53:58.724374-05:00,2018-01-19 12:54:10.738688-05:00
4,1,1,2018-01-19 12:54:15.587406-05:00,2018-01-19 12:54:27.621169-05:00


In [15]:
screen_timing

,SubjectID,Phase,Start,Stop
0,1,0,2018-01-19 12:52:42.999642-05:00,2018-01-19 12:52:55.195631-05:00
1,1,0,2018-01-19 12:53:02.695930-05:00,2018-01-19 12:53:15.302756-05:00
2,1,0,2018-01-19 12:53:22.896308-05:00,2018-01-19 12:53:34.942321-05:00
3,1,1,2018-01-19 12:53:58.724374-05:00,2018-01-19 12:54:10.738688-05:00
4,1,1,2018-01-19 12:54:15.587406-05:00,2018-01-19 12:54:27.621169-05:00
5,1,1,2018-01-19 12:54:32.739561-05:00,2018-01-19 12:54:44.729414-05:00
6,1,1,2018-01-19 12:54:51.248097-05:00,2018-01-19 12:55:03.278194-05:00
7,1,1,2018-01-19 12:55:08.072735-05:00,2018-01-19 12:55:20.089074-05:00
8,1,1,2018-01-19 12:55:28.847232-05:00,2018-01-19 12:55:40.869221-05:00
9,1,1,2018-01-19 12:55:47.165072-05:00,2018-01-19 12:55:59.184827-05:00


In [16]:
# Add the order sequence to the timing file
order = [i for i in range(1,4)] + [i for i in range(1,13)] + [i for i in range(1,3)] + [i for i in range(1, 12)]
order = order + order
screen_timing.insert(loc=2, column='Order', value=pd.Series(order, index=screen_timing.index))
screen_timing

,SubjectID,Phase,Order,Start,Stop
0,1,0,1,2018-01-19 12:52:42.999642-05:00,2018-01-19 12:52:55.195631-05:00
1,1,0,2,2018-01-19 12:53:02.695930-05:00,2018-01-19 12:53:15.302756-05:00
2,1,0,3,2018-01-19 12:53:22.896308-05:00,2018-01-19 12:53:34.942321-05:00
3,1,1,1,2018-01-19 12:53:58.724374-05:00,2018-01-19 12:54:10.738688-05:00
4,1,1,2,2018-01-19 12:54:15.587406-05:00,2018-01-19 12:54:27.621169-05:00
5,1,1,3,2018-01-19 12:54:32.739561-05:00,2018-01-19 12:54:44.729414-05:00
6,1,1,4,2018-01-19 12:54:51.248097-05:00,2018-01-19 12:55:03.278194-05:00
7,1,1,5,2018-01-19 12:55:08.072735-05:00,2018-01-19 12:55:20.089074-05:00
8,1,1,6,2018-01-19 12:55:28.847232-05:00,2018-01-19 12:55:40.869221-05:00
9,1,1,7,2018-01-19 12:55:47.165072-05:00,2018-01-19 12:55:59.184827-05:00


In [17]:
screen_timing.head()

,SubjectID,Phase,Order,Start,Stop
0,1,0,1,2018-01-19 12:52:42.999642-05:00,2018-01-19 12:52:55.195631-05:00
1,1,0,2,2018-01-19 12:53:02.695930-05:00,2018-01-19 12:53:15.302756-05:00
2,1,0,3,2018-01-19 12:53:22.896308-05:00,2018-01-19 12:53:34.942321-05:00
3,1,1,1,2018-01-19 12:53:58.724374-05:00,2018-01-19 12:54:10.738688-05:00
4,1,1,2,2018-01-19 12:54:15.587406-05:00,2018-01-19 12:54:27.621169-05:00


In [18]:
screen_timing.shape

(56, 5)

## Phase 1: Load Game Pair Order

In [19]:
pair_order = pd.read_csv('data/PairOrder.xls', sep='\t')
pair_order

,GamePairOrder,Period,SubjectID,Phase,Order,GamePairID
0,GamePairOrder,1,1,0,1,1
1,GamePairOrder,1,1,0,2,2
2,GamePairOrder,1,1,0,3,3
3,GamePairOrder,1,1,1,1,1
4,GamePairOrder,1,1,1,2,2
5,GamePairOrder,1,1,1,3,3
6,GamePairOrder,1,1,1,4,4
7,GamePairOrder,1,1,1,5,5
8,GamePairOrder,1,1,1,6,6
9,GamePairOrder,1,1,1,7,7


In [20]:
pair_order.shape

(56, 6)

#### Merge the Game Pair Order and Screen Timing

In [21]:
timing = pd.merge(pair_order, screen_timing, left_on=['SubjectID', 'Phase', 'Order'], right_on=['SubjectID', 'Phase', 'Order'])

In [22]:
timing.drop(['GamePairOrder'], axis=1, inplace=True)
timing

,Period,SubjectID,Phase,Order,GamePairID,Start,Stop
0,1,1,0,1,1,2018-01-19 12:52:42.999642-05:00,2018-01-19 12:52:55.195631-05:00
1,1,1,0,2,2,2018-01-19 12:53:02.695930-05:00,2018-01-19 12:53:15.302756-05:00
2,1,1,0,3,3,2018-01-19 12:53:22.896308-05:00,2018-01-19 12:53:34.942321-05:00
3,1,1,1,1,1,2018-01-19 12:53:58.724374-05:00,2018-01-19 12:54:10.738688-05:00
4,1,1,1,2,2,2018-01-19 12:54:15.587406-05:00,2018-01-19 12:54:27.621169-05:00
5,1,1,1,3,3,2018-01-19 12:54:32.739561-05:00,2018-01-19 12:54:44.729414-05:00
6,1,1,1,4,4,2018-01-19 12:54:51.248097-05:00,2018-01-19 12:55:03.278194-05:00
7,1,1,1,5,5,2018-01-19 12:55:08.072735-05:00,2018-01-19 12:55:20.089074-05:00
8,1,1,1,6,6,2018-01-19 12:55:28.847232-05:00,2018-01-19 12:55:40.869221-05:00
9,1,1,1,7,7,2018-01-19 12:55:47.165072-05:00,2018-01-19 12:55:59.184827-05:00


In [23]:
subject_1_timing = timing.loc[timing.SubjectID == 1].reset_index(drop=True)
subject_2_timing = timing.loc[timing.SubjectID == 2].reset_index(drop=True)

In [24]:
subject_1_timing

,Period,SubjectID,Phase,Order,GamePairID,Start,Stop
0,1,1,0,1,1,2018-01-19 12:52:42.999642-05:00,2018-01-19 12:52:55.195631-05:00
1,1,1,0,2,2,2018-01-19 12:53:02.695930-05:00,2018-01-19 12:53:15.302756-05:00
2,1,1,0,3,3,2018-01-19 12:53:22.896308-05:00,2018-01-19 12:53:34.942321-05:00
3,1,1,1,1,1,2018-01-19 12:53:58.724374-05:00,2018-01-19 12:54:10.738688-05:00
4,1,1,1,2,2,2018-01-19 12:54:15.587406-05:00,2018-01-19 12:54:27.621169-05:00
5,1,1,1,3,3,2018-01-19 12:54:32.739561-05:00,2018-01-19 12:54:44.729414-05:00
6,1,1,1,4,4,2018-01-19 12:54:51.248097-05:00,2018-01-19 12:55:03.278194-05:00
7,1,1,1,5,5,2018-01-19 12:55:08.072735-05:00,2018-01-19 12:55:20.089074-05:00
8,1,1,1,6,6,2018-01-19 12:55:28.847232-05:00,2018-01-19 12:55:40.869221-05:00
9,1,1,1,7,7,2018-01-19 12:55:47.165072-05:00,2018-01-19 12:55:59.184827-05:00


In [26]:
subject_2_timing

,Period,SubjectID,Phase,Order,GamePairID,Start,Stop
0,1,2,0,1,1,2018-01-19 12:52:38.717913-05:00,2018-01-19 12:52:50.781590-05:00
1,1,2,0,2,2,2018-01-19 12:52:57.790869-05:00,2018-01-19 12:53:09.877051-05:00
2,1,2,0,3,3,2018-01-19 12:53:14.907063-05:00,2018-01-19 12:53:27.482320-05:00
3,1,2,1,1,1,2018-01-19 12:53:50.192937-05:00,2018-01-19 12:54:02.278465-05:00
4,1,2,1,2,2,2018-01-19 12:54:10.127570-05:00,2018-01-19 12:54:22.191943-05:00
5,1,2,1,3,3,2018-01-19 12:54:27.218757-05:00,2018-01-19 12:54:39.296621-05:00
6,1,2,1,4,4,2018-01-19 12:54:45.792206-05:00,2018-01-19 12:54:57.847587-05:00
7,1,2,1,5,5,2018-01-19 12:55:02.582222-05:00,2018-01-19 12:55:14.641911-05:00
8,1,2,1,6,6,2018-01-19 12:55:23.343670-05:00,2018-01-19 12:55:35.421128-05:00
9,1,2,1,7,7,2018-01-19 12:55:41.686342-05:00,2018-01-19 12:55:53.744284-05:00


In [27]:
subject_1_timing.to_csv('data/subject_1_timing.csv')
subject_2_timing.to_csv('data/subject_2_timing.csv')